In [1]:
#hide
#default_exp showdoc

In [2]:
#export
# from nbdev.imports import *
# from nbdev.read import *
# from nbdev.export import *
from showdoc.lookup import *

import inspect,ast
# from fastcore.script import Param
from fastcore.all import *
from enum import Enum,EnumMeta
from textwrap import dedent

try: from IPython.display import Markdown,display
except ModuleNotFoundError: Markdown=None

In [3]:
from fastcore.test import *
import typing,numpy

# Showdoc

> Create documentation directly from python functions and classes

`showdoc` needs the following information to display for a symbol:

- the header level; this will be passed as a param
- the name
- the qualified name
- the prefix (`class`, `def`, or `enum`)
- link to the source if available
- the doc string
- the base class(es), if appropriate
- parameters, with types and defaults if provided

## Basic Details

In [4]:
#export
def _unwrapped_func(x):
    "Unwrap properties, typedispatch, and functools.wraps decorated functions"
    if hasattr(x,'first'): x = x.first()
    return getattr(getattr(x,'__wrapped__',x), "fget", x)

For testing we will use the following definitions:

In [5]:
def _f(f): return f

e = Enum('a',['b','c'])

class _T(int):
    "The class `_T`"
    def __init__(self, x:numpy.ndarray): ...
    def m(self, a:typing.Union[int,str]=0)->numpy.ndarray:
        "A method returning `numpy.ndarray`"
        return numpy.array([1])
    @_f
    def n(self): ...
    @property
    def t(self):
        "A property"
        ...

objs = L(e,_T,_T.m,_T.n,_T.t,max,typing.Union[int,str])
defs = L(e,_T,_f)

In [6]:
#export
def _name(o):
    o = _unwrapped_func(o)
    return str(try_attrs(o, '__name__', '__origin__', '_name')).split('.')[-1]

In [7]:
test_eq(objs.map(_name), ['a','_T','m','n','t','max','Union'])

In [8]:
#export
def qualname(o):
    o = _unwrapped_func(o)
    return getattr(o,'__qualname__', repr(o))

In [9]:
#export
def _code(o): return f'<code>{o}</code>'

def _qualname(o):
    o = _unwrapped_func(o)
    return _code(getattr(o,'__qualname__', repr(o)))

In [10]:
def _display_md(f, its): [display(Markdown(f(o))) for o in its]

In [11]:
#export
def nbdev_setting(mod, key, default=None):
    try: return nbdev_idx_mods[mod]['settings'][key]
    except KeyError: return default

In [12]:
#export
def sourcelink_url(o):
    "Source link to `o`"
    o = _unwrapped_func(o)
    try: line = inspect.getsourcelines(o)[1]
    except Exception: return None
    mod = o.__module__
    return f"{nbdev_setting(mod, 'git_url', '')}{mod.replace('.', '/')}.py#L{line}"

def _sourcelink(o):
    url = sourcelink_url(o)
    if url is None: return ''
    return f'<a href="{url}" class="source_link" style="float:right">[source]</a>'

In [13]:
sourcelink_url(NbdevLookup)

'https://github.com/fastai/showdoc/tree/master/showdoc/lookup.py#L26'

In [14]:
#export
def _docstring(o):
    res = inspect.getdoc(o)
    if not res: return ''
    if "\n\n" in res or "\n " in res: res = f"```\n{res}\n```"
    return res

def _basecls(o):
    res = getattr(o,'__bases__',[None])[0]
    if res: res = _name(res)
    return f" :: `{res}`" if res else ''

## Parameter list

In [15]:
#export
def typename(o):
    "Representation of type `t`"
    if getattr(o, '__args__', None): return str(o).split('.')[-1]
    res = _name(o)
    mod = getattr(o,'__module__','builtins')
    if mod=='builtins': return res
    return f"{mod}.{res}"

def _type_repr(t): return f":`{typename(t)}`" if t else ''

In [16]:
L(typing.Union[int,str],int,L).map(typename)

(#3) ['Union[int, str]','int','fastcore.foundation.L']

In [17]:
#export
def _param(p):
    _arg_prefixes = {inspect._VAR_POSITIONAL: '\*', inspect._VAR_KEYWORD:'\*\*'}
    arg_prefix = _arg_prefixes.get(p.kind, '') # asterisk prefix for *args and **kwargs
    res = f"**{arg_prefix}{_code(p.name)}**"
    res += _type_repr(empty2none(getattr(p,'annotation',None)))
    if p.default != p.empty:
        default = getattr(p.default, 'func', p.default) # partial
        res += f'=*`{getattr(default,"__name__",default)}`*'
    return res

def _args(x):
    "Formats function params to `param:Type=val` with markdown styling"
    try: sig = inspect.signature(x)
    except ValueError: return _code(re.search(r"(\([^)]*\))", x.__doc__).group(1)) # C functions
    except TypeError: return '' # properties

    fmt_params = [_param(v) for k,v in sig.parameters.items() if k not in ('self','cls')]
    res = f"({', '.join(fmt_params)})"
    ret = anno_dict(x).get('return',None)
    if ret: res += f" -> `{typename(ret)}`"
    return res

In [18]:
#export
@typedispatch
def format_showdoc(x:typing.Callable):
    "Markdown formatted version of `x`"
    return f'{_code("def")} {_qualname(x)}{_args(x)}'

@typedispatch
def format_showdoc(x): return _qualname(x)

In [19]:
Markdown(format_showdoc(_T.m))

<code>def</code> <code>_T.m</code>(**<code>a</code>**:`Union[int, str]`=*`0`*) -> `numpy.ndarray`

In [20]:
#export
@typedispatch
def format_showdoc(x:type):
    ar = _qualname(x)
    if inspect.isclass(x): ar = f"{_code('class')} {ar}"
    return ar + _args(x) + _basecls(x)

In [21]:
Markdown(format_showdoc(_T))

<code>class</code> <code>_T</code>(**<code>x</code>**:`numpy.ndarray`) :: `int`

In [22]:
#export
@typedispatch
def format_showdoc(x:(Enum,EnumMeta)):
    vals = ', '.join(L(x.__members__).map(_code("{}")))
    return f'{_code("enum")} = [{vals}]'

In [23]:
Markdown(format_showdoc(e))

<code>enum</code> = [<code>b</code>, <code>c</code>]

In [24]:
#export
_det_tmpl = """<details>
<summary>source</summary>

```python
{code}
```
</details>

"""

def show_sourcecode(o, maxlines=15):
    "Collapsible section showing source, without signature or docstring"
    try: src = inspect.getsource(o)
    except TypeError: return '' # builtin
    tree = ast.parse(dedent(src)).body[0]
    start,end = tree.body[0].lineno,tree.body[-1].end_lineno
    if end-start>maxlines: return '' # too big
    body_src = dedent('\n'.join(src.splitlines()[start:end]))
    return _det_tmpl.format(code=body_src)

In [25]:
Markdown(show_sourcecode(NbdevLookup))

<details>
<summary>source</summary>

```python
def __init__(self, strip_libs=None, incl_libs=None, skip_mods=None):
    skip_mods,strip_libs = setify(skip_mods),L(strip_libs)
    if incl_libs is not None: incl_libs = (L(incl_libs)+strip_libs).unique()
    self.entries = filter_keys(nbdev_idxs, lambda k: incl_libs is None or k in incl_libs)
    py_syms = merge(*L(o.modidx['syms'].values() for o in self.entries.values()).concat())
    for m in strip_libs:
        _d = self.entries[m].modidx
        stripped = {remove_prefix(k,f"{mod}."):v
                    for mod,dets in _d['syms'].items() if mod not in skip_mods
                    for k,v in dets.items()}
        py_syms = merge(stripped, py_syms)
    self.syms = py_syms

def __getitem__(self, s): return self.syms.get(s, None)
```
</details>



In [26]:
Markdown(show_sourcecode(NbdevLookup.linkify))

<details>
<summary>source</summary>

```python
in_fence=False
lines = md.splitlines()
for i,l in enumerate(lines):
    if l.startswith("```"): in_fence=not in_fence
    elif not l.startswith('    ') and not in_fence: lines[i] = self._link_line(l, L(skipped))
return '\n'.join(lines)
```
</details>



In [27]:
#export
def show_doc(elt, doc_string=True, name=None, title_level=None, disp=True, default_level=2):
    "Show documentation for element `elt`. Supported types: class, function, and enum."
    elt = getattr(elt, '__func__', elt)
    args = format_showdoc(elt)
    title_level = title_level or default_level
    doc =  f'<h{title_level} id="{_qualname(elt)}" class="doc_header">{_name(elt)}{_sourcelink(elt)}</h{title_level}>\n\n'
    if args: doc += f'> {args}\n\n'
    doc += show_sourcecode(elt) + _docstring(elt)
    doc = nbdev_lookup.linkify(doc)
    if disp:
        if Markdown: display(Markdown(doc))
        else: print(doc)
    else: return doc

In [28]:
objs[:-1].map(show_doc);

<h2 id="<code>a</code>" class="doc_header">a</h2>

> <code>enum</code> = [<code>b</code>, <code>c</code>]

An enumeration.

<h2 id="<code>_T</code>" class="doc_header">_T</h2>

> <code>class</code> <code>_T</code>(**<code>x</code>**:[`numpy.ndarray`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html#numpy.ndarray)) :: `int`

The class `_T`

<h2 id="<code>_T.m</code>" class="doc_header">m<a href="__main__.py#L8" class="source_link" style="float:right">[source]</a></h2>

> <code>_T.m</code>(**<code>a</code>**:`Union[int, str]`=*`0`*) -> [`numpy.ndarray`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html#numpy.ndarray)

<details>
<summary>source</summary>

```python
return numpy.array([1])
```
</details>

A method returning [`numpy.ndarray`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html#numpy.ndarray)

<h2 id="<code>_T.n</code>" class="doc_header">n<a href="__main__.py#L11" class="source_link" style="float:right">[source]</a></h2>

> <code>_T.n</code>()

<details>
<summary>source</summary>

```python

```
</details>


<h2 id="<code>_T.t</code>" class="doc_header">t<a href="__main__.py#L13" class="source_link" style="float:right">[source]</a></h2>

> <code>_T.t</code>

A property

<h2 id="<code>max</code>" class="doc_header">max</h2>

> <code>max</code><code>(iterable, *[, default=obj, key=func])</code>

```
max(iterable, *[, default=obj, key=func]) -> value
max(arg1, arg2, *args, *[, key=func]) -> value

With a single iterable argument, return its biggest item. The
default keyword-only argument specifies an object to return if
the provided iterable is empty.
With two or more arguments, return the largest argument.
```

### The doc command

In [29]:
def nbdev_module(sym, default=None):
    try: return nbdev_idx_mods[sym.__module__]['syms'][sym.__module__]
    except KeyError: return None

In [30]:
def nbdev_doclink(sym):
    nbmod = nbdev_module(sym)
    if not nbmod: return ''
    k = sym.__module__
    if not inspect.ismodule(sym): k += '.' + qualname(sym)
    return nbmod[k]

In [31]:
nbdev_doclink(NbdevLookup)

'https://showdoc.fast.ai/showdoc.lookup#NbdevLookup'

In [32]:
#export
def doc(elt):
    "Show `show_doc` info in preview window when used in a notebook"
    md = show_doc(elt, disp=False)
    doc_link = nbdev_doclink(elt)
    if doc_link is not None:
        md += f'\n\n<a href="{doc_link}" target="_blank" rel="noreferrer noopener">Show in docs</a>'
    display(Markdown(md))

In [33]:
doc(NbdevLookup)

<h2 id="<code>NbdevLookup</code>" class="doc_header">NbdevLookup<a href="https://github.com/fastai/showdoc/tree/master/showdoc/lookup.py#L26" class="source_link" style="float:right">[source]</a></h2>

> <code>class</code> <code>NbdevLookup</code>(**<code>strip_libs</code>**=*`None`*, **<code>incl_libs</code>**=*`None`*, **<code>skip_mods</code>**=*`None`*) :: `object`

<details>
<summary>source</summary>

```python
def __init__(self, strip_libs=None, incl_libs=None, skip_mods=None):
    skip_mods,strip_libs = setify(skip_mods),L(strip_libs)
    if incl_libs is not None: incl_libs = (L(incl_libs)+strip_libs).unique()
    self.entries = filter_keys(nbdev_idxs, lambda k: incl_libs is None or k in incl_libs)
    py_syms = merge(*L(o.modidx['syms'].values() for o in self.entries.values()).concat())
    for m in strip_libs:
        _d = self.entries[m].modidx
        stripped = {remove_prefix(k,f"{mod}."):v
                    for mod,dets in _d['syms'].items() if mod not in skip_mods
                    for k,v in dets.items()}
        py_syms = merge(stripped, py_syms)
    self.syms = py_syms

def __getitem__(self, s): return self.syms.get(s, None)
```
</details>

Mapping from symbol names to URLs with docs

<a href="https://showdoc.fast.ai/showdoc.lookup#NbdevLookup" target="_blank" rel="noreferrer noopener">Show in docs</a>

In [ ]:
from nbdev.doclinks import nbdev_build_lib
nbdev_build_lib()

In [ ]:
from nbdev.showdoc import show_doc

In [ ]:
show_doc(ExportModuleProcessor)